# **Практическая работа №4. Основы обработки пространственных данных в среде Apache Sedona**

## Задание №1. Найдите в открытых источниках или создайте самостоятельно тематический набор векторных данных:

In [1]:
%%capture
!pip install apache-sedona[spark]
!pip install findspark
!pip install -I shapely==1.8
!pip install geopandas==0.13.2

In [2]:
%%capture
!pip install pydeck
!pip install keplergl

In [3]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import LongType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.spark import *
from sedona.core.geom.envelope import Envelope

import findspark

findspark.init()

In [4]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.4_2.12:1.5.3,'
           'org.datasyslab:geotools-wrapper:1.5.3-28.2,'
           'uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.4'). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all'). \
    getOrCreate()

In [5]:
%%capture
'''
# Инициализация контекста Sedona
config = SedonaContext.builder() \
    # Установка пакетов JAR для Spark
    .config('spark.jars.packages',
            # Пакет Apache Sedona для работы с геопространственными данными в Spark
            'org.apache.sedona:sedona-spark-3.4_2.12:1.5.3,'
            # Обертка GeoTools для интеграции с Sedona
            'org.datasyslab:geotools-wrapper:1.5.3-28.2,'
            # Расширение GResearch для Spark, добавляющее дополнительные функции
            'uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.4'). \
    # Установка репозитория JAR, откуда будут загружаться пакеты
    .config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all'). \
    # Создание или получение существующего контекста Spark
    getOrCreate()
'''

In [6]:
# Создание контекста Sedona с использованием предварительно настроенного конфига Spark
sedona = SedonaContext.create(config)

# Получение контекста Spark из контекста Sedona
sc = sedona.sparkContext

# Установка системного свойства для глобальной кодировки в Sedona
sc.setSystemProperty("sedona.global.charset", "utf8")

In [13]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [15]:
shape_path = '/content/ne_10m_populated_places.shp'
ds = gpd.read_file(shape_path)
ds.head()

,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME,NAMEPAR,NAMEALT,NAMEASCII,ADM0CAP,CAPIN,WORLDCITY,MEGACITY,SOV0NAME,SOV_A3,ADM0NAME,ADM0_A3,ADM1NAME,ISO_A2,NOTE,LATITUDE,LONGITUDE,POP_MAX,POP_MIN,POP_OTHER,RANK_MAX,RANK_MIN,MEGANAME,LS_NAME,MAX_POP10,MAX_POP20,MAX_POP50,MAX_POP300,MAX_POP310,MAX_NATSCA,MIN_AREAKM,MAX_AREAKM,MIN_AREAMI,MAX_AREAMI,MIN_PERKM,MAX_PERKM,MIN_PERMI,MAX_PERMI,MIN_BBXMIN,MAX_BBXMIN,MIN_BBXMAX,MAX_BBXMAX,MIN_BBYMIN,MAX_BBYMIN,MIN_BBYMAX,MAX_BBYMAX,MEAN_BBXC,MEAN_BBYC,TIMEZONE,UN_FID,POP1950,POP1955,POP1960,POP1965,POP1970,POP1975,POP1980,POP1985,POP1990,POP1995,POP2000,POP2005,POP2010,POP2015,POP2020,POP2025,POP2050,MIN_ZOOM,WIKIDATAID,WOF_ID,CAPALT,NAME_EN,NAME_DE,NAME_ES,NAME_FR,NAME_PT,NAME_RU,NAME_ZH,LABEL,NAME_AR,NAME_BN,NAME_EL,NAME_HI,NAME_HU,NAME_ID,NAME_IT,NAME_JA,NAME_KO,NAME_NL,NAME_PL,NAME_SV,NAME_TR,NAME_VI,NE_ID,NAME_FA,NAME_HE,NAME_UK,NAME_UR,NAME_ZHT,GEONAMESID,FCLASS_ISO,FCLASS_US,FCLASS_FR,FCLASS_RU,FCLASS_ES,FCLASS_CN,FCLASS_TW,FCLASS_IN,FCLASS_NP,FCLASS_PK,FCLASS_DE,FCLASS_GB,FCLASS_BR,FCLASS_IL,FCLASS_PS,FCLASS_SA,FCLASS_EG,FCLASS_MA,FCLASS_PT,FCLASS_AR,FCLASS_JP,FCLASS_KO,FCLASS_VN,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,10,1,8.0,Admin-1 capital,Colonia del Sacramento,None,None,Colonia del Sacramento,0,None,0,0,Uruguay,URY,Uruguay,URY,Colonia,UY,None,-34.479999,-57.840003,21714,21714,0.0,7,7,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,America/Montevideo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,Q56064,421199749,0,Colonia del Sacramento,Colonia del Sacramento,Colonia del Sacramento,Colonia del Sacramento,Colónia do Sacramento,Колония-дель-Сакраменто,科洛尼亞·德爾·沙加緬度,None,كولونيا ديل ساكرامنتو,কোলোনিয়া দেল সাক্রামেন্তো,Κολόνια ντελ Σακραμέντο,कोलोनिया डेल सैक्रमेंटो,Colonia del Sacramento,Colonia del Sacramento,Colonia del Sacramento,コロニア・デル・サクラメント,콜로니아델사크라멘토,Colonia del Sacramento,Colonia del Sacramento,Colonia del Sacramento,Colonia del Sacramento,Colonia del Sacramento,1159112629,کولونیا دل ساکرامنتو,קולוניה דל סקרמנטו,Колонія-дель-Сакраменто,کولونیا دیل ساکرامینتو,科洛尼亞·德爾·沙加緬度,3443013.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-57.83612 -34.46979)
1,10,1,8.0,Admin-1 capital,Trinidad,None,None,Trinidad,0,None,0,0,Uruguay,URY,Uruguay,URY,Flores,UY,None,-33.543999,-56.900997,21093,21093,0.0,7,7,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,America/Montevideo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,Q862508,890444639,0,Trinidad,Trinidad,Trinidad,Trinidad,Trinidad,Тринидад,特立尼达,None,ترينيداد,ত্রিনিদাদ,Τρινιντάντ,त्रिनिडाड,Trinidad,Trinidad,Trinidad,トリニダ,트리니다드,Trinidad,Trinidad,Trinidad,Trinidad,Trinidad,1159112647,ترینیداد,טרינידד,Тринідад,ترینیداد,特立尼達,3439749.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-56.901 -33.544)
2,10,1,8.0,Admin-1 capital,Fray Bentos,None,None,Fray Bentos,0,None,0,0,Uruguay,URY,Uruguay,URY,Río Negro,UY,None,-33.138999,-58.303998,23279,23279,0.0,7,7,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,America/Montevideo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,Q849835,890451703,0,Fray Bentos,Fray Bentos,Fray Bentos,Fray Bentos,Fray Bentos,Фрай-Бентос,弗赖本托斯,None,فراي بنتوس,ফ্রায় বেন্তোস,Φρέι Μπέντος,फ्राई बैन्टोस,Fray Bentos,Fray Bentos,Fray Bentos,フライ・ベントス,프라이벤토스,Fray Bentos,Fray Bentos,Fray Bentos,Fray Bentos,Fray Bentos,1159112663,فری بنتوس,פראי בנטוס,Фрей Бентос,فرای بینتوس,弗賴本托斯,3442568.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

## Задание №2. Загрузите Ваш набор и преобразуйте его в Spark RDD/DataFrame

In [16]:
# Убираем колонку 'geometry' или преобразуем её в WKT (строку), чтобы Spark понимал
ds['geometry_wkt'] = ds['geometry'].apply(lambda geom: geom.wkt)

df_pandas = ds.drop(columns='geometry')


In [17]:
# Предполагаем, что SparkSession уже создан:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ShapefileToSpark").getOrCreate()

# Конвертация
df_spark = spark.createDataFrame(df_pandas)

# Проверка
df_spark.show(5)


+---------+--------+---------+---------------+--------------------+-------+-------+--------------------+-------+-----+---------+--------+--------+------+--------+-------+---------+------+----+----------+----------+-------+-------+---------+--------+--------+--------+-------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+---------+---------+------------------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+----------+---------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+-----------

In [82]:
ds = ds.copy()
ds['geometry_wkt'] = ds['geometry'].apply(lambda geom: geom.wkt)

columns_to_use = ['NAME', 'ADM0NAME','ADM0CAP', 'ADM0_A3', 'LATITUDE', 'LONGITUDE', 'POP_MAX', 'geometry_wkt']
populated_spark_df = spark.createDataFrame(ds[columns_to_use])
populated_spark_df.createOrReplaceTempView("populated_places")

## Задание №3. Сформируйте пять динамических пространственных запросов в контексте Вашего набора, опишите их предназначение и визуализируйте конечные выборки в среде KeplerGL

## №1 Все столицы мира

In [83]:
query_capitals = """
SELECT NAME, ADM0NAME, ADM0_A3, LATITUDE, LONGITUDE, POP_MAX, geometry_wkt
FROM populated_places
WHERE ADM0CAP = 1
"""
capitals_df = spark.sql(query_capitals)

capitals_df.show(5)

+------------+--------------------+-------+---------+----------+-------+--------------------+
|        NAME|            ADM0NAME|ADM0_A3| LATITUDE| LONGITUDE|POP_MAX|        geometry_wkt|
+------------+--------------------+-------+---------+----------+-------+--------------------+
|Vatican City|             Vatican|    VAT|41.903282| 12.453387|    832|POINT (12.4533865...|
|  San Marino|          San Marino|    SMR|43.936096|  12.44177|  29579|POINT (12.4417702...|
|       Vaduz|       Liechtenstein|    LIE|47.133724|   9.51667|  36281|POINT (9.5166695 ...|
|  Luxembourg|          Luxembourg|    LUX| 49.61166|  6.130003| 107260|POINT (6.1300028 ...|
|     Palikir|Federated States ...|    FSM| 6.916644|158.149974|   4645|POINT (158.149974...|
+------------+--------------------+-------+---------+----------+-------+--------------------+
only showing top 5 rows



In [84]:
from pyspark.sql.functions import expr

capitals_df = capitals_df.withColumn("geometry", expr("ST_GeomFromWKT(geometry_wkt)"))
capitals_df = capitals_df.drop("geometry_wkt")


In [85]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

sedona_pydeck_map = SedonaPyDeck.create_scatterplot_map(
    df=capitals_df,
    fill_color='[0, 0, 255]',
    radius_min_pixels=5,
    radius_max_pixels=30,
    radius_scale=5,
)

sedona_pydeck_map


<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 18.364749498579666,
    "longitude": 20.562890659365348,
    "zoom": 1
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "ADM0NAME": "Vatican",
          "ADM0_A3": "VAT",
          "LATITUDE": 41.903282,
          "LONGITUDE": 12.453387,
          "NAME": "Vatican City",
          "POP_MAX": 832,
          "coordinate_array_sedona": [
            12.4533865,
            41.9032822
          ],
          "geometry": {
            "coordinates": [
              12.4533865,
              41.9032822
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "San Marino",
          "ADM0_A3": "SMR",
          "LATITUDE": 43.936096,
          "LONGITUDE": 12.44177,
          "NAME": "San Marino",
          "POP_MAX": 29579,
          "coordinate_array_sedona": [
            12.4417702,
            43.9360958
          ],
          "geometry": {
            "coordinates": [
              12.4417702,
              43.9360958
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Liechtenstein",
          "ADM0_A3": "LIE",
          "LATITUDE": 47.133724,
          "LONGITUDE": 9.51667,
          "NAME": "Vaduz",
          "POP_MAX": 36281,
          "coordinate_array_sedona": [
            9.5166695,
            47.1337238
          ],
          "geometry": {
            "coordinates": [
              9.5166695,
              47.1337238
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Luxembourg",
          "ADM0_A3": "LUX",
          "LATITUDE": 49.61166,
          "LONGITUDE": 6.130003,
          "NAME": "Luxembourg",
          "POP_MAX": 107260,
          "coordinate_array_sedona": [
            6.1300028,
            49.6116604
          ],
          "geometry": {
            "coordinates": [
              6.1300028,
              49.6116604
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Federated States of Micronesia",
          "ADM0_A3": "FSM",
          "LATITUDE": 6.916644,
          "LONGITUDE": 158.149974,
          "NAME": "Palikir",
          "POP_MAX": 4645,
          "coordinate_array_sedona": [
            158.1499743,
            6.9166437
          ],
          "geometry": {
            "coordinates": [
              158.1499743,
              6.9166437
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Marshall Islands",
          "ADM0_A3": "MHL",
          "LATITUDE": 7.103004,
          "LONGITUDE": 171.38,
          "NAME": "Majuro",
          "POP_MAX": 25400,
          "coordinate_array_sedona": [
            171.3800002,
            7.1030043
          ],
          "geometry": {
            "coordinates": [
              171.3800002,
              7.1030043
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Tuvalu",
          "ADM0_A3": "TUV",
          "LATITUDE": -8.516652,
          "LONGITUDE": 179.216647,
          "NAME": "Funafuti",
          "POP_MAX": 4749,
          "coordinate_array_sedona": [
            179.2166471,
            -8.516652
          ],
          "geometry": {
            "coordinates": [
              179.2166471,
              -8.516652
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Palau",
          "ADM0_A3": "PLW",
          "LATITUDE": 7.487396,
          "LONGITUDE": 134.626549,
          "NAME": "Melekeok",
          "POP_MAX": 7026,
          "coordinate_array_sedona": [
            134.6265485,
            7.4873962
          ],
          "geometry": {
            "coordinates": [
              134.6265485,
              7.4873962
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Monaco",
          "ADM0_A3": 

## №2 Все города с населением > 5 млн чел.

In [86]:
query_megacities = """
SELECT NAME, ADM0NAME, ADM0_A3, LATITUDE, LONGITUDE, POP_MAX, geometry_wkt
FROM populated_places
WHERE POP_MAX > 5000000
"""

megacities_df = spark.sql(query_megacities)
megacities_df.show(5)


+----------------+--------------------+-------+---------+----------+-------+--------------------+
|            NAME|            ADM0NAME|ADM0_A3| LATITUDE| LONGITUDE|POP_MAX|        geometry_wkt|
+----------------+--------------------+-------+---------+----------+-------+--------------------+
|  Belo Horizonte|              Brazil|    BRA|-19.91308| -43.91695|5575000|POINT (-43.916950...|
|        Shenzhen|               China|    CHN|22.554316|114.120177|7581000|POINT (114.061153...|
|    Philadelphia|United States of ...|    USA|40.001919|-75.171942|5492000|POINT (-75.179841...|
|Ho Chi Minh City|             Vietnam|    VNM|10.781971|106.693081|5314000|POINT (106.702984...|
|          Lahore|            Pakistan|    PAK|31.561917| 74.348079|6577000|POINT (74.3480789...|
+----------------+--------------------+-------+---------+----------+-------+--------------------+
only showing top 5 rows



In [36]:
from pyspark.sql.functions import expr

megacities_df = megacities_df.withColumn("geometry", expr("ST_GeomFromWKT(geometry_wkt)"))
megacities_df = megacities_df.drop("geometry_wkt")


In [38]:
sedona_pydeck_map = SedonaPyDeck.create_scatterplot_map(
    df=megacities_df,
    fill_color='[255, 0, 0]',
    radius_min_pixels=5,
    radius_max_pixels=30,
    radius_scale=5,
)

sedona_pydeck_map


<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 20.236327998383615,
    "longitude": 38.41679575757584,
    "zoom": 1
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "ADM0NAME": "Brazil",
          "ADM0_A3": "BRA",
          "LATITUDE": -19.91308,
          "LONGITUDE": -43.91695,
          "NAME": "Belo Horizonte",
          "POP_MAX": 5575000,
          "coordinate_array_sedona": [
            -43.9169504,
            -19.9130802
          ],
          "geometry": {
            "coordinates": [
              -43.9169504,
              -19.9130802
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "China",
          "ADM0_A3": "CHN",
          "LATITUDE": 22.554316,
          "LONGITUDE": 114.120177,
          "NAME": "Shenzhen",
          "POP_MAX": 7581000,
          "coordinate_array_sedona": [
            114.06115351896277,
            22.54809707412332
          ],
          "geometry": {
            "coordinates": [
              114.06115351896277,
              22.54809707412332
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "United States of America",
          "ADM0_A3": "USA",
          "LATITUDE": 40.001919,
          "LONGITUDE": -75.171942,
          "NAME": "Philadelphia",
          "POP_MAX": 5492000,
          "coordinate_array_sedona": [
            -75.1798412402816,
            39.94594243742075
          ],
          "geometry": {
            "coordinates": [
              -75.1798412402816,
              39.94594243742075
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Vietnam",
          "ADM0_A3": "VNM",
          "LATITUDE": 10.781971,
          "LONGITUDE": 106.693081,
          "NAME": "Ho Chi Minh City",
          "POP_MAX": 5314000,
          "coordinate_array_sedona": [
            106.70298470218748,
            10.762124565177512
          ],
          "geometry": {
            "coordinates": [
              106.70298470218748,
              10.762124565177512
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Pakistan",
          "ADM0_A3": "PAK",
          "LATITUDE": 31.561917,
          "LONGITUDE": 74.348079,
          "NAME": "Lahore",
          "POP_MAX": 6577000,
          "coordinate_array_sedona": [
            74.3480789,
            31.5619174
          ],
          "geometry": {
            "coordinates": [
              74.3480789,
              31.5619174
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Pakistan",
          "ADM0_A3": "PAK",
          "LATITUDE": 24.871938,
          "LONGITUDE": 66.988063,
          "NAME": "Karachi",
          "POP_MAX": 12130000,
          "coordinate_array_sedona": [
            66.9880631,
            24.8719381
          ],
          "geometry": {
            "coordinates": [
              66.9880631,
              24.8719381
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "China",
          "ADM0_A3": "CHN",
          "LATITUDE": 29.566923,
          "LONGITUDE": 106.593036,
          "NAME": "Chongqing",
          "POP_MAX": 6461000,
          "coordinate_array_sedona": [
            106.5930358,
            29.5669229
          ],
          "geometry": {
            "coordinates": [
              106.5930358,
              29.5669229
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "China",
          "ADM0_A3": "CHN",
          "LATITUDE": 23.146927,
          "LONGITUDE": 113.323064,
          "NAME": "Guangzhou",
          "POP_MAX": 8829000,
          "coordinate_array_sedona": [
            113.26178516712943,
            23.12472343296695
          ],
          "geometry": {
            "coordinates": [
              113.26178516712943,
            

## №3 Города в пределах 100 км от Москвы

In [87]:
query_near_moscow = """
SELECT NAME, ADM0NAME, ADM0_A3, LATITUDE, LONGITUDE, POP_MAX, geometry_wkt
FROM populated_places
WHERE ST_DistanceSphere(ST_GeomFromWKT(geometry_wkt),
                        ST_Point(37.6173, 55.7558)) < 100000
"""

near_moscow_df = spark.sql(query_near_moscow)
near_moscow_df.show(5)


+--------------+--------+-------+---------+---------+-------+--------------------+
|          NAME|ADM0NAME|ADM0_A3| LATITUDE|LONGITUDE|POP_MAX|        geometry_wkt|
+--------------+--------+-------+---------+---------+-------+--------------------+
|Orekhovo-Zuevo|  Russia|    RUS|55.820015|38.979987| 140247|POINT (38.9799873...|
|          Klin|  Russia|    RUS|56.343058|36.698731|  80778|POINT (36.6987312...|
|Sergiyev Posad|  Russia|    RUS| 56.33001|38.170011| 109252|POINT (38.1700109...|
|       Obninsk|  Russia|    RUS|55.080448|36.620028| 107392|POINT (36.620028 ...|
|    Yegoryevsk|  Russia|    RUS|55.384796| 39.02939|  89795|POINT (39.02939 5...|
+--------------+--------+-------+---------+---------+-------+--------------------+
only showing top 5 rows



In [48]:
from pyspark.sql.functions import expr

near_moscow_df = near_moscow_df.withColumn("geometry", expr("ST_GeomFromWKT(geometry_wkt)"))
near_moscow_df = near_moscow_df.drop("geometry_wkt")

In [ ]:
# в датасете содержатся не все города (только крупные, значимые административные центры, столицы и т.п.).

In [49]:
sedona_pydeck_map = SedonaPyDeck.create_scatterplot_map(
    df=near_moscow_df,
    fill_color='[255, 165, 0]',  # оранжевые точки
    radius_min_pixels=5,
    radius_max_pixels=30,
    radius_scale=5,
)

sedona_pydeck_map

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 55.70743639,
    "longitude": 37.7532578,
    "zoom": 7
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 55.820015,
          "LONGITUDE": 38.979987,
          "NAME": "Orekhovo-Zuevo",
          "POP_MAX": 140247,
          "coordinate_array_sedona": [
            38.9799873,
            55.8200153
          ],
          "geometry": {
            "coordinates": [
              38.9799873,
              55.8200153
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 56.343058,
          "LONGITUDE": 36.698731,
          "NAME": "Klin",
          "POP_MAX": 80778,
          "coordinate_array_sedona": [
            36.6987312,
            56.3430583
          ],
          "geometry": {
            "coordinates": [
              36.6987312,
              56.3430583
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 56.33001,
          "LONGITUDE": 38.170011,
          "NAME": "Sergiyev Posad",
          "POP_MAX": 109252,
          "coordinate_array_sedona": [
            38.1700109,
            56.33001
          ],
          "geometry": {
            "coordinates": [
              38.1700109,
              56.33001
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 55.080448,
          "LONGITUDE": 36.620028,
          "NAME": "Obninsk",
          "POP_MAX": 107392,
          "coordinate_array_sedona": [
            36.620028,
            55.080448
          ],
          "geometry": {
            "coordinates": [
              36.620028,
              55.080448
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 55.384796,
          "LONGITUDE": 39.02939,
          "NAME": "Yegoryevsk",
          "POP_MAX": 89795,
          "coordinate_array_sedona": [
            39.02939,
            55.3847964
          ],
          "geometry": {
            "coordinates": [
              39.02939,
              55.3847964
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 55.38043,
          "LONGITUDE": 37.529947,
          "NAME": "Podolsk",
          "POP_MAX": 320635,
          "coordinate_array_sedona": [
            37.5299467,
            55.3804297
          ],
          "geometry": {
            "coordinates": [
              37.5299467,
              55.3804297
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 56.180691,
          "LONGITUDE": 36.980885,
          "NAME": "Solnechnogorsk",
          "POP_MAX": 58891,
          "coordinate_array_sedona": [
            36.9808846,
            56.1806909
          ],
          "geometry": {
            "coordinates": [
              36.9808846,
              56.1806909
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 55.870426,
          "LONGITUDE": 38.480018,
          "NAME": "Noginsk",
          "POP_MAX": 229731,
          "coordinate_array_sedona": [
            38.4800179,
            55.8704256
          ],
          "geometry": {
            "coordinates": [
              38.4800179,
              55.8704256
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Russia",
          "ADM0_A3": "RUS",
          "LATITUDE": 54.93038,
        

## №4 Все города Европы

In [88]:
query_europe_cities = """
SELECT NAME, ADM0NAME, ADM0_A3, LATITUDE, LONGITUDE, POP_MAX, geometry_wkt
FROM populated_places
WHERE LATITUDE BETWEEN 35 AND 72
  AND LONGITUDE BETWEEN -10 AND 40
"""

europe_df = spark.sql(query_europe_cities)
europe_df.show(5)


+-----------+--------+-------+---------+---------+-------+--------------------+
|       NAME|ADM0NAME|ADM0_A3| LATITUDE|LONGITUDE|POP_MAX|        geometry_wkt|
+-----------+--------+-------+---------+---------+-------+--------------------+
|   L'Ariana| Tunisia|    TUN|36.866673|10.199998|  97687|POINT (10.1999975...|
|   Jendouba| Tunisia|    TUN|36.500004| 8.749999|  51408|POINT (8.7499986 ...|
|  Kasserine| Tunisia|    TUN|35.216703| 8.716699|  76243|POINT (8.7166985 ...|
|Sdid Bouzid| Tunisia|    TUN|35.016696| 9.500005|  42098|POINT (9.5000045 ...|
|    Siliana| Tunisia|    TUN|36.083304| 9.383302|  26960|POINT (9.3833016 ...|
+-----------+--------+-------+---------+---------+-------+--------------------+
only showing top 5 rows



In [71]:
from pyspark.sql.functions import expr

europe_df = europe_df.withColumn("geometry", expr("ST_GeomFromWKT(geometry_wkt)"))
europe_df = europe_df.drop("geometry_wkt")


In [72]:
sedona_pydeck_map = SedonaPyDeck.create_scatterplot_map(
    df=europe_df,
    fill_color='[0, 128, 255]',
    radius_min_pixels=5,
    radius_max_pixels=30,
    radius_scale=5,
)

sedona_pydeck_map


<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 48.26259731016055,
    "longitude": 16.57939660251843,
    "zoom": 2
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "ADM0NAME": "Tunisia",
          "ADM0_A3": "TUN",
          "LATITUDE": 36.866673,
          "LONGITUDE": 10.199998,
          "NAME": "L'Ariana",
          "POP_MAX": 97687,
          "coordinate_array_sedona": [
            10.1999975,
            36.8666732
          ],
          "geometry": {
            "coordinates": [
              10.1999975,
              36.8666732
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Tunisia",
          "ADM0_A3": "TUN",
          "LATITUDE": 36.500004,
          "LONGITUDE": 8.749999,
          "NAME": "Jendouba",
          "POP_MAX": 51408,
          "coordinate_array_sedona": [
            8.7499986,
            36.5000041
          ],
          "geometry": {
            "coordinates": [
              8.7499986,
              36.5000041
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Tunisia",
          "ADM0_A3": "TUN",
          "LATITUDE": 35.216703,
          "LONGITUDE": 8.716699,
          "NAME": "Kasserine",
          "POP_MAX": 76243,
          "coordinate_array_sedona": [
            8.7166985,
            35.2167031
          ],
          "geometry": {
            "coordinates": [
              8.7166985,
              35.2167031
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Tunisia",
          "ADM0_A3": "TUN",
          "LATITUDE": 35.016696,
          "LONGITUDE": 9.500005,
          "NAME": "Sdid Bouzid",
          "POP_MAX": 42098,
          "coordinate_array_sedona": [
            9.5000045,
            35.0166961
          ],
          "geometry": {
            "coordinates": [
              9.5000045,
              35.0166961
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Tunisia",
          "ADM0_A3": "TUN",
          "LATITUDE": 36.083304,
          "LONGITUDE": 9.383302,
          "NAME": "Siliana",
          "POP_MAX": 26960,
          "coordinate_array_sedona": [
            9.3833016,
            36.0833041
          ],
          "geometry": {
            "coordinates": [
              9.3833016,
              36.0833041
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Tunisia",
          "ADM0_A3": "TUN",
          "LATITUDE": 35.483913,
          "LONGITUDE": 11.040877,
          "NAME": "Mahdia",
          "POP_MAX": 45977,
          "coordinate_array_sedona": [
            11.0408766,
            35.483913
          ],
          "geometry": {
            "coordinates": [
              11.0408766,
              35.483913
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Tunisia",
          "ADM0_A3": "TUN",
          "LATITUDE": 35.730702,
          "LONGITUDE": 10.767295,
          "NAME": "Monastir",
          "POP_MAX": 71546,
          "coordinate_array_sedona": [
            10.7672946,
            35.7307021
          ],
          "geometry": {
            "coordinates": [
              10.7672946,
              35.7307021
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Tunisia",
          "ADM0_A3": "TUN",
          "LATITUDE": 36.399996,
          "LONGITUDE": 10.146997,
          "NAME": "Zaghouan",
          "POP_MAX": 16911,
          "coordinate_array_sedona": [
            10.1469966,
            36.3999962
          ],
          "geometry": {
            "coordinates": [
              10.1469966,
              36.3999962
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Turkey",
          "ADM0_A3": "TUR",
          "LATITUDE": 41.742999,
  

## №5 Города, у которых названия начинаются с "San", "Santa", "Saint"

In [89]:
query_saint_cities = """
SELECT NAME, ADM0NAME, ADM0_A3, LATITUDE, LONGITUDE, POP_MAX, geometry_wkt
FROM populated_places
WHERE NAME LIKE 'San%' OR NAME LIKE 'Santa%' OR NAME LIKE 'Saint%'
"""
saint_df = spark.sql(query_saint_cities)
saint_df.show(10)


+--------------------+------------------+-------+---------+----------+-------+--------------------+
|                NAME|          ADM0NAME|ADM0_A3| LATITUDE| LONGITUDE|POP_MAX|        geometry_wkt|
+--------------------+------------------+-------+---------+----------+-------+--------------------+
|       Santa Bárbara|          Honduras|    HND|14.919003|   -88.236|  15119|POINT (-88.235999...|
|        Saint Gallen|       Switzerland|    CHE|47.422998|  9.361999|  70572|POINT (9.3619986 ...|
|            Santarém|          Portugal|    PRT|   39.231| -8.682003|  29385|POINT (-8.6820026...|
|        Sanniquellie|           Liberia|    LBR|    7.371|    -8.685|  11415|POINT (-8.6849995...|
|San Francisco Gotera|       El Salvador|    SLV|13.699998|-88.100001|  16152|POINT (-88.100000...|
|         San Vicente|       El Salvador|    SLV|13.641003|   -88.785|  37326|POINT (-88.784999...|
|     Saint Ann's Bay|           Jamaica|    JAM|18.432639|-77.199525|  13671|POINT (-77.199524...|


In [78]:
from pyspark.sql.functions import expr

saint_df = saint_df.withColumn("geometry", expr("ST_GeomFromWKT(geometry_wkt)"))
saint_df = saint_df.drop("geometry_wkt")


In [79]:
sedona_pydeck_map = SedonaPyDeck.create_scatterplot_map(
    df=saint_df,
    fill_color='[0, 200, 100]',
    radius_min_pixels=5,
    radius_max_pixels=30,
    radius_scale=5,
)

sedona_pydeck_map

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "latitude": 5.149225129834182,
    "longitude": -60.950940453347314,
    "zoom": 1
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "data": [
        {
          "ADM0NAME": "Honduras",
          "ADM0_A3": "HND",
          "LATITUDE": 14.919003,
          "LONGITUDE": -88.236,
          "NAME": "Santa B\u00e1rbara",
          "POP_MAX": 15119,
          "coordinate_array_sedona": [
            -88.2359996,
            14.919003
          ],
          "geometry": {
            "coordinates": [
              -88.2359996,
              14.919003
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Switzerland",
          "ADM0_A3": "CHE",
          "LATITUDE": 47.422998,
          "LONGITUDE": 9.361999,
          "NAME": "Saint Gallen",
          "POP_MAX": 70572,
          "coordinate_array_sedona": [
            9.3619986,
            47.4229981
          ],
          "geometry": {
            "coordinates": [
              9.3619986,
              47.4229981
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Portugal",
          "ADM0_A3": "PRT",
          "LATITUDE": 39.231,
          "LONGITUDE": -8.682003,
          "NAME": "Santar\u00e9m",
          "POP_MAX": 29385,
          "coordinate_array_sedona": [
            -8.6820026,
            39.2310001
          ],
          "geometry": {
            "coordinates": [
              -8.6820026,
              39.2310001
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Liberia",
          "ADM0_A3": "LBR",
          "LATITUDE": 7.371,
          "LONGITUDE": -8.685,
          "NAME": "Sanniquellie",
          "POP_MAX": 11415,
          "coordinate_array_sedona": [
            -8.6849995,
            7.3710001
          ],
          "geometry": {
            "coordinates": [
              -8.6849995,
              7.3710001
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "El Salvador",
          "ADM0_A3": "SLV",
          "LATITUDE": 13.699998,
          "LONGITUDE": -88.100001,
          "NAME": "San Francisco Gotera",
          "POP_MAX": 16152,
          "coordinate_array_sedona": [
            -88.1000005,
            13.6999981
          ],
          "geometry": {
            "coordinates": [
              -88.1000005,
              13.6999981
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "El Salvador",
          "ADM0_A3": "SLV",
          "LATITUDE": 13.641003,
          "LONGITUDE": -88.785,
          "NAME": "San Vicente",
          "POP_MAX": 37326,
          "coordinate_array_sedona": [
            -88.7849996,
            13.641003
          ],
          "geometry": {
            "coordinates": [
              -88.7849996,
              13.641003
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Jamaica",
          "ADM0_A3": "JAM",
          "LATITUDE": 18.432639,
          "LONGITUDE": -77.199525,
          "NAME": "Saint Ann's Bay",
          "POP_MAX": 13671,
          "coordinate_array_sedona": [
            -77.1995245,
            18.4326391
          ],
          "geometry": {
            "coordinates": [
              -77.1995245,
              18.4326391
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Guatemala",
          "ADM0_A3": "GTM",
          "LATITUDE": 14.966001,
          "LONGITUDE": -91.8,
          "NAME": "San Marcos",
          "POP_MAX": 25088,
          "coordinate_array_sedona": [
            -91.7999995,
            14.9660011
          ],
          "geometry": {
            "coordinates": [
              -91.7999995,
              14.9660011
            ],
            "type": "Point"
          }
        },
        {
          "ADM0NAME": "Dominican